<a href="https://colab.research.google.com/github/yamaguchinorimitsu/ajax_app/blob/master/%E3%82%B9%E3%83%83%E3%82%AD%E3%83%AA%E6%A9%9F%E6%A2%B0%E7%AC%AC%EF%BC%99%E7%AB%A0%E7%AD%94%E3%81%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
%matplotlib inline 

df = pd.read_csv('/content/Bank.csv')
print(df.shape)
df.head()

(27128, 16)


,id,age,job,marital,education,default,amount,housing,loan,contact,day,month,duration,campaign,previous,y
0,1,39,blue-collar,married,secondary,no,1756.0,yes,no,cellular,3,apr,370.055237,1,0,1
1,2,51,entrepreneur,married,primary,no,1443.0,no,no,cellular,18,feb,233.998933,10,0,1
2,3,36,management,single,tertiary,no,436.0,no,no,cellular,13,apr,NaN,1,2,0
3,4,63,retired,married,secondary,no,474.0,no,no,cellular,25,jan,252.525808,1,0,0
4,5,31,management,single,tertiary,no,354.0,no,no,cellular,30,apr,NaN,1,2,0


In [2]:
# まず、ダミー変数化をしたいが、文字列の列が複数あるので抜き出す。
str_col_name=['job','default','marital','education','housing','loan','contact','month']
str_df = df[str_col_name]
#複数列を一気にダミー変数化
str_df2=pd.get_dummies(str_df,drop_first=True)

num_df = df.drop(str_col_name,axis=1)#数値列を抜き出す
df2 = pd.concat([num_df,str_df2,str_df],axis=1)#結合(今後の集計の利便性も考慮してstr_dfも結合しておく)

In [3]:
df2.columns

Index(['id', 'age', 'amount', 'day', 'duration', 'campaign', 'previous', 'y',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'default_yes', 'marital_married', 'marital_single',
       'education_secondary', 'education_tertiary', 'education_unknown',
       'housing_yes', 'loan_yes', 'contact_sending _document',
       'contact_telephone', 'month_aug', 'month_dec', 'month_feb', 'month_jan',
       'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov',
       'month_oct', 'month_sep', 'job', 'default', 'marital', 'education',
       'housing', 'loan', 'contact', 'month'],
      dtype='object')

In [4]:
#訓練&検証データとテストデータに分割
train_val,test = train_test_split(df2,test_size=0.1,random_state=9)
train_val.head()

,id,age,amount,day,duration,campaign,previous,y,job_blue-collar,job_entrepreneur,...,month_oct,month_sep,job,default,marital,education,housing,loan,contact,month
13378,13379,46,0.0,10,378.293875,1,0,1,0,0,...,0,0,unemployed,no,married,tertiary,yes,no,cellular,jul
3800,3801,41,-333.0,30,300.845752,3,2,0,1,0,...,0,0,blue-collar,no,divorced,secondary,yes,no,cellular,jan
10398,10399,30,5389.0,6,159.053623,3,0,1,0,0,...,0,0,management,no,single,tertiary,no,no,cellular,aug
23401,23402,39,255.0,5,334.802583,1,0,0,0,0,...,0,0,management,no,single,tertiary,yes,no,sending _document,may
26223,26224,47,33.0,5,NaN,1,0,1,0,0,...,0,0,services,no,single,secondary,yes,no,cellular,may


In [5]:
a=train_val.isnull().sum()
a[a>1]

duration    6331
dtype: int64

In [6]:
train_val['duration'].median()

314.89135074820206

In [7]:
#とりあえず最初は中央値で補完
train_val2= train_val.fillna(train_val.median())

<ipython-input-7-9e2d5178499f>:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_val2= train_val.fillna(train_val.median())


In [8]:
#不均衡なデータであるか確認
train_val2['y'].value_counts()

0    16601
1     7814
Name: y, dtype: int64

In [9]:
#とりあえず、全ての特徴量を利用してモデルを作ってみる。
t =train_val2['y']
x = train_val2.drop(str_col_name,axis=1)
x =x.drop(['id','y','day'],axis=1)

x_train,x_val,y_train,y_val = train_test_split(x,t,test_size=0.2,random_state=13)

#不均衡データに対応できるように、class_weight引数も設定
model = tree.DecisionTreeClassifier(random_state=3,max_depth=3,class_weight='balanced')

#class_weightを指定しないとちなみに正解率は0.7ぐらい
#model = tree.DecisionTreeClassifier(random_state=3,max_depth=5)
model.fit(x_train,y_train)
model.score(x_val,y_val)

0.6493958631988531

In [10]:
# def learn(x,t,depth=3):
#     x_train,x_test,y_train,y_test = train_test_split(x,t,test_size = 0.2,random_state = 0)
#     model = tree.DecisionTreeClassifier(max_depth =depth,random_state = 0,class_weight="balanced")
#     model.fit(x_train,y_train)

#     #訓練データでも正解率を出している
#     score=model.score(X=x_train,y=y_train)

#     #テストデータで正解率を出している
#     score2=model.score(X=x_test,y=y_test)
#     return round(score,3),round(score2,3),model



# for j in range(1,20): # jは木の深さ jには1～14が入る
#     # xは特徴量、tは正解データ
#     train_score,test_score,model = learn(x,t,depth = j)
#     sentence="訓練データの正解率{}"
#     sentence2="テストデータの正解率{}"
#     total_sentence='深さ{}:'+sentence+sentence2
#     print(total_sentence.format(j,train_score,test_score))


In [11]:
#まず、さくっと学習できるようなlearn関数を定義する。
def learn(x,t,i):
    x_train,x_val,y_train,y_val = train_test_split(x,t,test_size=0.2,random_state=13)

    datas=[x_train,x_val,y_train,y_val]
    #不均衡データに対応できるように、class_weight引数も設定
    model = tree.DecisionTreeClassifier(random_state=i,max_depth=i,class_weight='balanced')
    model.fit(x_train,y_train)
    train_score=model.score(x_train,y_train)
    
    val_score=model.score(x_val,y_val)
    return train_score,val_score,model,datas

    #とりあえず、for文で様々な木の深さでの正解率を調べてみる
for i in range(1,20):
    s1,s2,model,datas = learn(x,t,i)
    print(i,s1,s2)

1 0.6995187384804423 0.6993651443784559
2 0.6492422690968667 0.6493958631988531
3 0.6492422690968667 0.6493958631988531
4 0.7357669465492525 0.7481056727421667
5 0.7767765717796437 0.7773909481875896
6 0.80124923202949 0.8023755887773909
7 0.8224452181036248 0.8095433135367602
8 0.8448699569936514 0.8238787630554987
9 0.858386237968462 0.8230595945115707
10 0.8756399754249437 0.829817734998976
11 0.8987814867909072 0.8363710833503993
12 0.910147450337907 0.8269506450952283
13 0.9284251484742986 0.8236739709195167
14 0.943170182265001 0.8124104034405079
15 0.9587343845996313 0.820397296743805
16 0.9684620110587754 0.8156870776162195
17 0.9776264591439688 0.8177349989760393
18 0.9845893917673562 0.8134343641204178
19 0.9900163833708786 0.8152774933442556


In [12]:
#深さ11以降はvalが停滞するので過学習が発生ここでテストデータでチェック
model = tree.DecisionTreeClassifier(max_depth=11,random_state=11)
model.fit(x,t)
test2 = test.copy()
test2=test2.fillna(train_val.median())

test_y=test2['y']
test_x = test2.drop(str_col_name,axis=1)
test_x =test_x.drop(['id','y','day'],axis=1)
model.score(test_x,test_y)

<ipython-input-12-16e81aa1e0c5>:5: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  test2=test2.fillna(train_val.median())


0.8330261702911905

In [12]:
model.feature_importances_

array([1.43315746e-01, 3.96812086e-02, 5.29793362e-01, 1.03042282e-01,
       1.57441172e-02, 1.89964590e-03, 1.30908318e-03, 2.19315742e-03,
       2.40999335e-03, 7.97533559e-04, 1.11527096e-03, 2.54012417e-03,
       1.25782518e-03, 1.56103337e-03, 2.43834955e-03, 2.45315633e-04,
       1.34849859e-03, 1.84850581e-03, 1.90339098e-03, 2.09766847e-03,
       2.46703665e-03, 1.27069046e-03, 6.49505171e-02, 4.17388960e-02,
       1.45361561e-02, 1.78548128e-03, 2.83889354e-03, 3.38443272e-04,
       1.10739861e-03, 2.71660427e-04, 2.56367272e-03, 2.73496962e-03,
       2.57382988e-03, 1.99502593e-03, 1.22091207e-03, 6.21851543e-04,
       4.42452326e-04])

In [13]:
#特徴量重要度の確認（上とは違うコード）(ちなみに、連続量とダミー変数を比べると、連続量の方が重要度は高め出力されてしまう事に注意)
a=pd.Series(model.feature_importances_,index=x.columns).sort_values(ascending=False)
a[0:9]#campaignやdurationが重要度が大きいと分かる

duration                     0.529793
age                          0.143316
campaign                     0.103042
housing_yes                  0.064951
loan_yes                     0.041739
amount                       0.039681
previous                     0.015744
contact_sending _document    0.014536
month_aug                    0.002839
dtype: float64

In [14]:
#durationの欠損値の処理を行う
# durationに関係がありそうなものを見つけるためには7章の様に集計処理で判断したい。
#よって、集計処理の軸になるstr_dfの列名をいったん確認
print(str_df.columns)

Index(['job', 'default', 'marital', 'education', 'housing', 'loan', 'contact',
       'month'],
      dtype='object')


In [15]:
for name in str_df.columns:
    print(train_val.groupby(name)['y'].mean())
    print("next=========")

job
admin.           0.304550
blue-collar      0.352886
entrepreneur     0.390453
housemaid        0.290043
management       0.307299
retired          0.369268
self-employed    0.301974
services         0.337611
student          0.166667
technician       0.307423
unemployed       0.241084
unknown          0.294118
Name: y, dtype: float64
next=========
default
no     0.321336
yes    0.249428
Name: y, dtype: float64
next=========
marital
divorced    0.334059
married     0.341211
single      0.269086
Name: y, dtype: float64
next=========
education
primary      0.351921
secondary    0.318327
tertiary     0.301592
unknown      0.354354
Name: y, dtype: float64
next=========
housing
no     0.217959
yes    0.400910
Name: y, dtype: float64
next=========
loan
no     0.300195
yes    0.424328
Name: y, dtype: float64
next=========
contact
cellular             0.277883
sending _document    0.416372
telephone            0.311842
Name: y, dtype: float64
next=========
month
apr    0.312070
aug    0.251

In [16]:
#housing. loan. contactあたりが関係してそうだ。
#ピボットテーブルを作成する

print(pd.pivot_table(train_val,index="housing",columns="loan",values="duration"))
print(pd.pivot_table(train_val,index="housing",columns="contact",values="duration"))
print(pd.pivot_table(train_val,index="loan",columns="contact",values="duration"))

loan             no         yes
housing                        
no       237.256887  301.180261
yes      332.635840  439.472217
contact    cellular  sending _document   telephone
housing                                           
no       249.821962         249.179202  272.785529
yes      348.532350         349.179816  368.638141
contact    cellular  sending _document   telephone
loan                                              
no       297.588613         314.969378  304.960446
yes      375.465455         400.911698  379.000771


In [20]:
#今回はloan housingの２×２のものを使用する
def nan_fill(train_val):
    isnull = train_val['duration'].isnull()

    #train_valを変えたくないので、py基本文法のcopyメソッドでコピーを作って
    #train_val2に代入
    train_val2 = train_val.copy()
    #housing=yesの処理
    train_val2.loc[(isnull) & (train_val2['housing']=='yes') 
                  &(train_val2['loan']=='yes'),'duration' ]=439
    train_val2.loc[(isnull) & (train_val2['housing']=='yes') 
                  &(train_val2['loan']=='no'),'duration' ]=332

    #housing=noの処理
    train_val2.loc[(isnull) & (train_val2['housing']=='no') 
                  &(train_val2['loan']=='yes'),'duration' ]=301
    train_val2.loc[(isnull) & (train_val2['housing']=='no') 
                  &(train_val2['loan']=='no'),'duration' ]=237
    
    return train_val2
train_val2=nan_fill(train_val)


#  train_val2.loc[(isnull) & (train_val2['housing']=='yes') 
#　　　　&(train_val2['loan']=='yes'),'duration' ]=439 

#という行は、duration 列が欠損している場合かつ条件 
#(isnull) & (train_val2['housing']=='yes') &(train_val2['loan']=='yes') が満たされる場合に、
#その欠損箇所に値 439 を代入しています。

#この処理は、特定の条件に基づいて欠損値を補完するためのものです。
#housing 列が 'yes' であり、loan 列が 'yes' である場合に限定して欠損値を 439 で補完しています。



In [21]:
#特徴量重要度が大きかった項目とyの関係

train_val2.groupby('y')['duration'].median()

y
0    263.575464
1    346.960871
Name: duration, dtype: float64

In [22]:
train_val2.groupby('y')['amount'].median()

y
0    449.0
1    449.0
Name: amount, dtype: float64

In [23]:
train_val2.groupby('y')['campaign'].median()

y
0    2.0
1    3.0
Name: campaign, dtype: float64

In [24]:
train_val2.groupby('y')['age'].median()

y
0    38.0
1    41.0
Name: age, dtype: float64

In [25]:
#上記よりdurationの値が大きければy=1になる確率が上がりそうだということがわかる

#ここでいったん、learn関数で分析

t =train_val2['y']

x = train_val2.drop(str_col_name,axis=1)
x =x.drop(['id','y','day'],axis=1)
#x =train_val2[xcol]
for i in range(1,20):
    s1,s2,model,datas = learn(x,t,i)
    print(i,s1,s2)

1 0.7462113454843334 0.7511775547818964
2 0.7462113454843334 0.7511775547818964
3 0.7507167724759369 0.7554781896375179
4 0.7481056727421667 0.7569117345893918
5 0.7746262543518329 0.7751382346917879
6 0.7780565226295311 0.7712471841081302
7 0.8182469793159942 0.8117960270325619
8 0.8275138234691788 0.8077001843129223
9 0.8483514233053451 0.8273602293671923
10 0.8692402211755068 0.8335039934466516
11 0.8782510751587139 0.8251075158713905
12 0.8999590415728036 0.8261314765513005
13 0.914294491091542 0.8281793979111202
14 0.9298074953921769 0.8220356338316609
15 0.9457812819987712 0.8220356338316609
16 0.9606287118574647 0.820602088879787
17 0.9706123284865861 0.8171206225680934
18 0.9785992217898832 0.8187589596559492
19 0.9862277288552119 0.8199877124718411


In [24]:
#どのような間違い方をしているのか確認
s1,s2,model,datas = learn(x,t,10)

#訓練データでの予測結果と実際の値の2軸で個数集計flagがFalseならば、検証データで集計
def syuukei(model,datas,flag=False):
    if flag:
        pre=model.predict(datas[0])
        y_val=datas[2]
    else:
        pre=model.predict(datas[1])
        y_val=datas[3]
    data={
        "pred":pre,
        "true":y_val
    }
    tmp=pd.DataFrame(data)
    return tmp,pd.pivot_table(tmp,index="true",columns="pred",values="true",aggfunc=len)
tmp,a=syuukei(model,datas,False)
a

pred,0,1
true,,
0,2796,587
1,226,1274
